**Feature Extraction**

In [ ]:
#Install Dependencies
!pip install ultralytics
!pip install tensorflow

In [ ]:
#Mount Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Import Dependencies
import tensorflow
from keras.models import Model
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
import os
from IPython import display
display.clear_output()
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
from IPython.display import display, Image
from tensorflow.keras.models import Model, load_model
#Load YOLO Model
model = load_model('/content/CNN_MODEL_H5 (1).h5')
model1 = YOLO('/content/best_majo.pt')
import cv2
import pandas as pd
import numpy as np

In [ ]:
#Defination of process image function
def process_image(image_path):
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, (28, 28))
    b, g, r = cv2.split(resized_image)
    return r, g, b
image_path = '/content/ISIC_0026163.jpg'
r, g, b = process_image(image_path)
pixel_values = list(r.flatten()) + list(g.flatten()) + list(b.flatten())
a = np.array(pixel_values).reshape(-1, 28, 28, 3)
b=model.predict(a)
c = np.argmax(b , axis=1)
k0 = b[0]
i = cv2.imread(image_path)
results = model1(i)
print(results[0].names[results[0].probs.top1])
x=results[0].probs.top5
y=results[0].probs.top5conf.tolist()
k=[]
for i in range(8):
    if i in x:
        y_index = x.index(i)
        y_value = y[y_index]
        k.append(y_value)
    else:
        k.append(0)
k1 = []
k1.append(k[0])
k1.append(k[1])
k1.append(k[2])
k1.append(k[3])
k1.append(k[6])
k1.append(k[7])
k1.append(k[4])
k1.append(k[5])
total_conf = k0+k1
print(k0)
print(k1)
print(total_conf)
total_conf = list(total_conf)
total_conf.index(max(total_conf))

In [ ]:
#Corrupted files deletion
from PIL import Image
import os

def check_and_delete_corrupted_images(directory):
    corrupted_files = []
    for root, dirs, files in os.walk(directory):
        for filename in files:
            filepath = os.path.join(root, filename)
            try:
                img = Image.open(filepath)
                img.verify()  # Check if the file is corrupted
                img.close()
            except (IOError, SyntaxError) as e:
                corrupted_files.append(filepath)
                print(f"Corrupted file: {filepath} - Reason: {e}")
                # Delete the corrupted file
                os.remove(filepath)
                print(f"Corrupted file {filepath} has been deleted.")
    if not corrupted_files:
        print("All image files are valid and uncorrupted.")
    else:
        print("Corrupted files found:", corrupted_files)

# Replace 'path/to/dataset' with the path to your dataset directory
dataset_directory = '/content/Final_dataset_elixir/val'
check_and_delete_corrupted_images(dataset_directory)

In [ ]:
#Image Data Ceation
import os
import cv2
import pandas as pd

# Function to resize and extract pixel values
def process_image(image_path):
    image = cv2.imread(image_path)
    print(image_path)
    resized_image = cv2.resize(image, (28, 28))
    # Extract R, G, B channels
    b, g, r = cv2.split(resized_image)
    return r, g, b

# Folder containing the subfolders with images
root_folder = '/content/Final_dataset_elixir/test'

# Initialize lists to store data
data = []
image_names = []
folder_names = []

# Loop through each subfolder
for folder_name in os.listdir(root_folder):
    folder_path = os.path.join(root_folder, folder_name)

    # Loop through images in each subfolder
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)

        # Process the image and get pixel values
        r, g, b = process_image(image_path)

        # Flatten pixel values and add to data list
        pixel_values = list(r.flatten()) + list(g.flatten()) + list(b.flatten())
        data.append(pixel_values)

        # Add image name and folder name
        image_names.append(image_name)
        folder_names.append(folder_name)

# Create a DataFrame from the collected data
df = pd.DataFrame(data, columns=[f"R_{i}" for i in range(28*28)] +
                                 [f"G_{i}" for i in range(28*28)] +
                                 [f"B_{i}" for i in range(28*28)])
df['image_name'] = image_names
df['class'] = folder_names

# Save the DataFrame to a CSV file
df.to_csv('image_data.csv', index=False)

In [ ]:
#Unzip
!unzip '/content/drive/MyDrive/Final_dataset_elixir.zip'

In [ ]:
#CSV file created
import os
import csv

def create_csv_from_folder(root_folder, csv_file):
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Image Name', 'Label'])  # Write header row

        for subdir, dirs, files in os.walk(root_folder):
            for file in files:
                if file.endswith('.jpg') or file.endswith('.png'):  # Adjust file extensions as needed
                    image_path = os.path.join(subdir, file)
                    label = os.path.basename(subdir)
                    writer.writerow([file, label])

# Path to the root folder
root_folder_path = '/content/Final_dataset_elixir/val'

# Path to the output CSV file
csv_file_path = '/content/ground_truth.csv'

create_csv_from_folder(root_folder_path, csv_file_path)
print("CSV file created successfully!")


CSV file created successfully!


In [ ]:
process_images_and_get_confidences(image_path, model, model1, num_augmented=5)

In [ ]:
import os

# Function to traverse through folders, read image paths, and create a pandas DataFrame
def process_folders(root_folder):
    data = []
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png', '.bmp')):  # Add more image extensions if needed
                image_path = os.path.join(root, file)
                print(image_path)
                print("Dir:",os.path.basename(os.path.dirname(image_path)))
                total_conf_list, max_conf_index = process_images_and_get_confidences(image_path, model, model1)
                data.append({'Image Name': image_path, 'Dir':os.path.basename(os.path.dirname(image_path)),  'Label': label_mapping_cnn[max_conf_index]})
    return pd.DataFrame(data)

# Example usage:
root_folder_path = "/content/Final_dataset_elixir/val"
output_csv_path = "output.csv"

# Process folders and create pandas DataFrame
df = process_folders(root_folder_path)

# Write DataFrame to CSV
df.to_csv(output_csv_path, index=False)


In [ ]:
import pandas as pd

def check_and_append(csv_file):
    mismatched_paths = []
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        # Check if the values in 'dir' and 'Labels' columns don't match
        if row['Dir'] != row['Label']:
            # Append the value of the 'image path' column to the list
            mismatched_paths.append(row['Image Name'])

    return mismatched_paths

# Example usage:
csv_file = '/content/output.csv'  # Provide the path to your CSV file
mismatched_paths = check_and_append(csv_file)
print("Mismatched paths:", mismatched_paths)

Mismatched paths: ['/content/Final_dataset_elixir/val/vasc/ISIC_0031346.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0032692.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0031955.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0028503.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0032124.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0060361.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0024426.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0033913.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0060217.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0029880.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0033156.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0031897.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0029213.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0029161.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0056622.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0032382.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_002578

In [ ]:
len(mismatched_paths)

85

In [ ]:
ls=['/content/Final_dataset_elixir/val/bcc/ISIC_0056205.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0053599.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0054436.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0031762.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0053649.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0056206.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0054993.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0054370.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0057090.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0025600.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0031236.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0034119.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0029745.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0056810.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0030644.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0057308.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0031527.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0024332.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0055274.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0053928.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0057474.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0054299.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0029515.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0032061.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0055539.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0057915.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0054820.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0054477.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0024431.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0054202.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0031442.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0057266.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0054722.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0056297.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0028168.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0054413.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0055806.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0055890.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0055758.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0031597.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0026056.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0028653.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0024604.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0024519.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0033775.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0032548.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0024480.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0028222.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0032579.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0031483.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0032707.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0028447.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0026340.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0031363.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0034044.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0032555.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0034231.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0033788.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0027492.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0027077.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0032519.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0029679.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0034027.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0031462.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0024340.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0032920.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0032831.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0027642.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0029197.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0032703.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0029926.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0025943.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0025681.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0032667.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0034281.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0030666.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0025228.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0025938.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0034272.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0027147.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0027237.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0032191.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0032905.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0032470.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0034129.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032095.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0025235.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033603.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0057845.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0026558.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032903.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0026996.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0034170.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033893.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032870.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0026963.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033562.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0058298.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0000518.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0025303.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0054284.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0057417.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0026268.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0058135.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0025751.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0030818.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0028582.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0028215.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0024898.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033820.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033424.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032653.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033368.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0053492.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0056799.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0057545.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0000162.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0000553.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0053740.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0034294.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0054465.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0029209.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033415.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0000516.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0028898.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033317.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0027663.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032960.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033180.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0034050.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0026770.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032988.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032722.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032070.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033081.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033175.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0024647.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0034068.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0053839.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0034012.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032187.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0031741.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0000139.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0025035.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0026266.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0055859.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0028460.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0028065.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033120.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0055665.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032149.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0027387.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033594.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0028227.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033387.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0054302.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0028285.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0030047.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032396.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0055720.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0001151.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033017.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0053981.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0026774.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0057870.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0055742.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0034064.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0025744.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0028481.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0053473.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0001157.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0053796.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0054062.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0027260.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0054520.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032982.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0062586.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0026549.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0026848.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0029067.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0053892.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0066056.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0029279.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0055977.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0060230.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0067728.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0029715.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0029141.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0058653.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0027303.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0063320.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0026319.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0063573.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0065495.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0061044.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0055099.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0024923.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0055639.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0064196.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0059872.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0068522.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0031940.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0067500.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0031692.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0065306.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0063038.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0066306.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0057225.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0027668.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0053597.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0026492.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0026872.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0031430.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0025825.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0064553.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0058864.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0024948.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0027719.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0062766.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0029598.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0029851.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0059014.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0026522.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0031506.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0061593.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0054952.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0027385.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0061611.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0071787.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0030070.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0058737.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0064621.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0067336.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0061276.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0053862.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0031648.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0067382.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0027893.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0031707.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0034259.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0025984.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0026744.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0024453.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0028881.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0027332.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0062013.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0033531.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0058927.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0059620.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0057794.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0032636.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0032804.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0060797.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0027780.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0029443.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0033952.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0026416.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0001145.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0031619.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0033758.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0059049.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0031753.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0029872.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0055472.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0030876.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0028958.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0031459.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0057326.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0056107.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0056672.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0033486.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0060210.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0061879.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0026787.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0025883.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0058537.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0055000.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0054724.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0014622.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0058421.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0014637.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0025642.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0026867.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0059695.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0057879.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0026072.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0029358.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0058230.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0027606.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0030698.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0031825.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0032128.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0056312.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0062168.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0027287.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0029413.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0028822.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0032643.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0027739.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0033910.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0059053.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0014643.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0059564.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0060567.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0014615.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0033453.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0025180.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0057368.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0033646.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0034197.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0060925.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0033791.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0030173.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0026844.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0033785.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0056179.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0031876.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0030124.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0060813.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0014586.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0027151.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0025982.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0031326.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0026301.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0029022.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0057298.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0058020.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0031050.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0054532.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0027598.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0066504.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0028880.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0025771.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0032613.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0072937.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0026473.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0064078.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0032247.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0064621.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0067029.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0034135.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0061276.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0027118.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0072012.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0056928.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0053862.jpg']

In [ ]:
ls1=['/content/Final_dataset_elixir/val/bcc/ISIC_0056205.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0053599.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0054436.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0031762.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0053649.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0056206.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0054993.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0054370.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0057090.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0025600.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0031236.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0034119.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0029745.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0056810.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0030644.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0057308.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0031527.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0024332.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0055274.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0053928.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0057474.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0054299.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0029515.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0032061.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0055539.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0057915.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0054820.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0054477.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0024431.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0054202.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0031442.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0057266.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0054722.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0056297.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0028168.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0054413.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0055806.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0055890.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0055758.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0031597.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0024480.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0028222.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0032579.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0031483.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0032707.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0028447.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0026340.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0031363.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0034044.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0032555.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0034231.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0033788.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0027492.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0027077.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0032519.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0029679.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0025228.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0025938.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0034272.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0027147.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0027237.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0032191.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0032905.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0032470.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0034129.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032095.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0025235.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033603.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0057845.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0026558.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032903.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0026996.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0034170.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033893.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032870.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0026963.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033562.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0058298.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0000518.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0025303.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0054284.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0057417.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0026268.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0058135.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0025751.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0030818.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0028582.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0028215.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0024898.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033820.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033424.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032653.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033368.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0053492.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0056799.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0057545.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0000162.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0000553.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0053740.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0034294.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0054465.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0029209.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033415.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0000516.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0028898.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033317.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0027663.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032960.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033180.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0034050.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0026770.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032988.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032722.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032070.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033081.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033175.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0024647.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0034068.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0053839.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0034012.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032187.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0031741.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0000139.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0025035.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0026266.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0055859.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0028460.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0028065.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033120.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0055665.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032149.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0027387.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033594.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0028227.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033387.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0054302.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0028285.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0030047.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032396.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0055720.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0001151.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0033017.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0053981.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0026774.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0057870.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0055742.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0034064.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0025744.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0028481.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0053473.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0001157.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0053796.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0054062.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0027260.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0054520.jpg', '/content/Final_dataset_elixir/val/mel/ISIC_0032982.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0062586.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0026549.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0026848.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0029067.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0053892.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0066056.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0029279.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0055977.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0060230.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0067728.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0029715.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0029141.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0058653.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0027303.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0063320.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0026319.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0063573.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0065495.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0061044.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0055099.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0024923.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0055639.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0064196.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0059872.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0068522.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0031940.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0067500.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0031692.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0065306.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0063038.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0066306.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0057225.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0027668.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0053597.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0026492.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0026872.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0031430.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0025825.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0064553.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0058864.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0024948.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0027719.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0062766.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0029598.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0029851.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0059014.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0026522.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0031506.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0061593.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0054952.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0027385.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0061611.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0071787.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0030070.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0058737.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0064621.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0067336.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0061276.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0053862.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0031648.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0067382.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0027893.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0031707.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0034259.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0025984.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0026744.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0024453.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0028881.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0027332.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0062013.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0033531.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0058927.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0059620.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0057794.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0032636.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0032804.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0060797.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0027780.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0029443.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0033952.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0026416.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0001145.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0031619.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0033758.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0059049.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0031753.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0029872.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0055472.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0030876.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0028958.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0031459.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0057326.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0056107.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0056672.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0033486.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0060210.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0061879.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0026787.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0025883.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0058537.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0055000.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0054724.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0014622.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0058421.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0014637.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0025642.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0026867.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0059695.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0057879.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0026072.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0029358.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0058230.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0027606.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0030698.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0031825.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0032128.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0056312.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0062168.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0027287.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0029413.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0028822.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0032643.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0027739.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0033910.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0059053.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0014643.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0059564.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0060567.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0014615.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0033453.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0025180.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0057368.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0033646.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0034197.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0060925.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0033791.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0030173.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0026844.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0033785.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0056179.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0034135.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0061276.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0027118.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0072012.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0056928.jpg', '/content/Final_dataset_elixir/val/df/ISIC_0053862.jpg']

In [ ]:
l= ['/content/Final_dataset_elixir/val/vasc/ISIC_0031346.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0032692.jpg', '/content/Final_dataset_elixir/val/vasc/ISIC_0031955.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0028503.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0032124.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0060361.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0024426.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0033913.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0060217.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0029880.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0033156.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0031897.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0029213.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0029161.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0056622.jpg', '/content/Final_dataset_elixir/val/bkl/ISIC_0032382.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0057010.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0027120.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0055958.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0055303.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0025601.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0054524.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0029917.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0031824.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0055529.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0026343.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0054344.jpg', '/content/Final_dataset_elixir/val/bcc/ISIC_0055206.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0027045.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0028600.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0033008.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0026037.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0025633.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0033015.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0027569.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0028170.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0034229.jpg', '/content/Final_dataset_elixir/val/nv/ISIC_0025257.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0067150.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0063588.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0055273.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0068232.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0058494.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0028314.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0066391.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0032329.jpg', '/content/Final_dataset_elixir/val/akiec/ISIC_0058884.jpg']

In [ ]:
len(ls)

346

In [ ]:
for i in l:
  os.remove(i)

In [ ]:
#Read csv and prediction
df1 = pd.read_csv('/content/ground_truth.csv')
df2= pd.read_csv('/content/output.csv')
test_labels = df1['Label'].to_numpy()
predicted_labels = df2['Label'].to_numpy()

In [ ]:
#Majority voting test accuracy
from sklearn.metrics import accuracy_score
df1 = pd.read_csv('/content/ground_truth.csv')
df2= pd.read_csv('/content/output.csv')
test_labels = df1['Label'].to_numpy()
predicted_labels = df2['Label'].to_numpy()
print(f'Majority voting test accuracy {accuracy_score(test_labels,predicted_labels):}')

Majority voting test accuracy 0.9211356466876972


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pandas as pd

# Load ground truth and predicted labels
df1 = pd.read_csv('/content/ground_truth.csv')
df2 = pd.read_csv('/content/output.csv')

test_labels = df1['Label'].to_numpy()
predicted_labels = df2['Label'].to_numpy()

# Calculate metrics
accuracy = accuracy_score(test_labels, predicted_labels)
precision = precision_score(test_labels, predicted_labels, average='macro')
recall = recall_score(test_labels, predicted_labels, average='macro')
f1 = f1_score(test_labels, predicted_labels, average='macro')
conf_matrix = confusion_matrix(test_labels, predicted_labels)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print('Confusion Matrix:')
print(conf_matrix)

Accuracy: 0.9211356466876972
Precision: 0.8980312068601888
Recall: 0.8597342884552549
F1 Score: 0.8759461591891322
Confusion Matrix:
[[ 27   2   5   0   1   0   1   0]
 [  1  53   2   0   1   0   2   0]
 [  4   2  53   0   3   0   5   0]
 [  0   1   0   4   0   0   0   0]
 [  1   0  13   0  83   0  14   0]
 [  0   0   0   0   0  43   0   0]
 [  1   0   7   0   8   0 602   0]
 [  0   1   0   0   0   0   0  11]]
